In [3]:
import pandas as pd
import math


In [4]:
!kaggle datasets download -d gauravmalik26/food-delivery-dataset

Dataset URL: https://www.kaggle.com/datasets/gauravmalik26/food-delivery-dataset
License(s): other
 51%|███████████████████▍                  | 1.00M/1.95M [00:00<00:00, 1.78MB/s]
100%|██████████████████████████████████████| 1.95M/1.95M [00:00<00:00, 3.03MB/s]


In [5]:
!unzip food-delivery-dataset.zip

Archive:  food-delivery-dataset.zip
  inflating: Sample_Submission.csv   
  inflating: test.csv                
  inflating: train.csv               


In [142]:
data = pd.read_csv('train.csv').dropna().drop_duplicates()

print(data.columns)

data_of_interest = data[["Restaurant_longitude", "Restaurant_latitude","Delivery_location_longitude", "Delivery_location_latitude","Road_traffic_density", "Vehicle_condition","Type_of_vehicle","Weatherconditions", "Time_taken(min)"]]

# data_of_interest.loc[data_of_interest["Type_of_vehicle"] == "scooter"]

#data_of_interest

# t = data.query('Type_of_vehicle == "scooter"')
# t = data[data['Type_of_vehicle'].str.contains('scooter')]
# t = data[data['Type_of_vehicle'].str.strip() == 'scooter']
# t = data_of_interest[data_of_interest['Type_of_vehicle'].str.strip() == 'scooter']
# t = data_of_interest['Time_taken(min)'].str.replace('(min)','').str.strip()
# t = data[['Restaurant_longitude', 'Restaurant_latitude', 'Delivery_location_latitude','Delivery_location_longitude']]
data

Index(['ID', 'Delivery_person_ID', 'Delivery_person_Age',
       'Delivery_person_Ratings', 'Restaurant_latitude',
       'Restaurant_longitude', 'Delivery_location_latitude',
       'Delivery_location_longitude', 'Order_Date', 'Time_Orderd',
       'Time_Order_picked', 'Weatherconditions', 'Road_traffic_density',
       'Vehicle_condition', 'Type_of_order', 'Type_of_vehicle',
       'multiple_deliveries', 'Festival', 'City', 'Time_taken(min)'],
      dtype='object')


,ID,Delivery_person_ID,Delivery_person_Age,Delivery_person_Ratings,Restaurant_latitude,Restaurant_longitude,Delivery_location_latitude,Delivery_location_longitude,Order_Date,Time_Orderd,Time_Order_picked,Weatherconditions,Road_traffic_density,Vehicle_condition,Type_of_order,Type_of_vehicle,multiple_deliveries,Festival,City,Time_taken(min)
0,0x4607,INDORES13DEL02,37,4.9,22.745049,75.892471,22.765049,75.912471,19-03-2022,11:30:00,11:45:00,conditions Sunny,High,2,Snack,motorcycle,0,No,Urban,(min) 24
1,0xb379,BANGRES18DEL02,34,4.5,12.913041,77.683237,13.043041,77.813237,25-03-2022,19:45:00,19:50:00,conditions Stormy,Jam,2,Snack,scooter,1,No,Metropolitian,(min) 33
2,0x5d6d,BANGRES19DEL01,23,4.4,12.914264,77.678400,12.924264,77.688400,19-03-2022,08:30:00,08:45:00,conditions Sandstorms,Low,0,Drinks,motorcycle,1,No,Urban,(min) 26
3,0x7a6a,COIMBRES13DEL02,38,4.7,11.003669,76.976494,11.053669,77.026494,05-04-2022,18:00:00,18:10:00,conditions Sunny,Medium,0,Buffet,motorcycle,1,No,Metropolitian,(min) 21
4,0x70a2,CHENRES12DEL01,32,4.6,12.972793,80.249982,13.012793,80.289982,26-03-2022,13:30:00,13:45:00,conditions Cloudy,High,1,Snack,scooter,1,No,Metropolitian,(min) 30
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45588,0x7c09,JAPRES04DEL01,30,4.8,26.902328,75.794257,26.912328,75.804257,24-03-2022,11:35:00,11:45:00,conditions Windy,High,1,Meal,motorcycle,0,No,Metropolitian,(min) 32
45589,0xd641,AGRRES16DEL01,21,4.6,0.000000,0.000000,0.070000,0.070000,16-02-2022,19:55:00,20:10:00,conditions Windy,Jam,0,Buffet,motorcycle,1,No,Metropolitian,(min) 36
45590,0x4f8d,CHENRES08DEL03,30,4.9,13.022394,80.242439,13.052394,80.272439,11-03-2022,23:50:00,00:05:00,conditions Cloudy,Low,1,Drinks,scooter,0,No,Metropolitian,(min) 16
45591,0x5eee,COIMBRES11DEL01,20,4.7,11.001753,76.986241,11.041753,77.026241,07-03-2022,13:35:00,13:40:00,conditions Cloudy,High,0,Snack,motorcycle,1,No,Metropolitian,(min) 26


In [143]:
def calc_dist_with_haversine(coord1: object, coord2: object):

    # Coordinates in decimal degrees (e.g. 2.89078, 12.79797)
    lon1, lat1 = coord1
    lon2, lat2 = coord2

    R = 6371000  # radius of Earth in meters
    phi_1 = math.radians(lat1)
    phi_2 = math.radians(lat2)

    delta_phi = math.radians(lat2 - lat1)
    delta_lambda = math.radians(lon2 - lon1)

    a = math.sin(delta_phi / 2.0) ** 2 + math.cos(phi_1) * math.cos(phi_2) * math.sin(delta_lambda / 2.0) ** 2
    
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))

    meters = R * c  # output distance in meters
    km = meters / 1000.0  # output distance in kilometers

    meters = round(meters, 3)
    return round(km, 3)


restaurant_location = data[['Restaurant_longitude', 'Restaurant_latitude']].loc[1]
delivery_location = data[['Delivery_location_latitude','Delivery_location_longitude']].loc[1]

calc_dist_with_haversine(restaurant_location, delivery_location)


8022.464

In [154]:
def calc_restaurant_to_delivery_distance(row):
    return calc_dist_with_haversine(row[['Restaurant_longitude', 'Restaurant_latitude']], row[['Delivery_location_latitude','Delivery_location_longitude']])


data_of_interest["distance_km"] = data.apply(calc_restaurant_to_delivery_distance, axis=1)
data_of_interest["Time_taken(min)"] = pd.to_numeric(data_of_interest["Time_taken(min)"].str.replace('(min)',''), errors="coerce")

scooter = data_of_interest.loc[data_of_interest["Type_of_vehicle"].str.strip() == "scooter"]


# t = scooter.loc[scooter["Type_of_vehicle"].str.strip() == '']
pd.set_option('display.max_rows', scooter.shape[0]+1)

# pd.to_numeric(scooter["Time_taken(min)"], errors="coerce").isna()
scooter["Time_taken(min)"]

/tmp/ipykernel_78603/3438930387.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_of_interest["distance_km"] = data.apply(calc_restaurant_to_delivery_distance, axis=1)
/tmp/ipykernel_78603/3438930387.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_of_interest["Time_taken(min)"] = pd.to_numeric(data_of_interest["Time_taken(min)"].str.replace('(min)',''), errors="coerce")


1        33
4        30
6        40
11       21
13       41
15       33
22       39
26       38
32       25
33       35
34       30
36       10
37       19
38       11
39       11
42       33
44       22
45       16
47       25
49       18
51       23
57       49
58       20
60       18
61       22
62       34
64       17
67       15
68       16
69       36
70       18
71       17
74       19
78       44
80       42
82       39
87       19
92       15
93       15
99       39
101      19
106      22
107      42
108      12
111      15
114      24
121      39
122      18
124      18
125      18
126      49
129      23
130      11
136      17
140      19
143      34
145      23
148      35
152      19
153      27
160      15
163      22
169      26
170      26
172      19
173      18
176      17
177      11
187      25
199      11
203      26
209      13
212      29
218      29
219      18
228      29
229      25
231      24
236      19
237      20
238      25
253      26
257      17
260 

In [170]:
def average_speed_by_distance_and_time(total_kilometers: float, total_time: float):
    return total_kilometers / total_time

def calc_restaurant_to_delivery_distance(row):
    return calc_dist_with_haversine(row[['Restaurant_longitude', 'Restaurant_latitude']], row[['Delivery_location_latitude','Delivery_location_longitude']])

#mangler fra restaurant og til destination?
#her er det tiden vi skal have fat i og ikke gennemsnit hastighed.

# def estimate_time_by_estimators(total_distance: calc_dist_with_haversine, weather: str, traffic: str, type_of_vechicle: str, vehicle_condition: str):
#     scooter = data[data['Type_of_vehicle'].str.strip() == 'scooter']
#     return average_speed_by_distance_and_time(total_distance, scooter.sum)

def estimate_time_by_estimators(total_distance: float):
    scooter = data_of_interest[data_of_interest['Type_of_vehicle'].str.strip() == 'scooter']
    return average_speed_by_distance_and_time(scooter['distance_km'].sum(), scooter['Time_taken(min)'].sum())


# def estimate_time_by_estimators(total_distance: float):
#     scooter = data_of_interest[data_of_interest['Type_of_vehicle'].str.strip() == 'scooter']
#     return scooter['Time_taken(min)'].sum()

distance = data_of_interest['distance_km'][0]


scooter = data_of_interest[data_of_interest['Type_of_vehicle'].str.strip() == 'scooter']

t = scooter['Time_taken(min)'].sum(min_count=1)

tq = estimate_time_by_estimators(distance)

tq

271.0685971259573